#Scikic API v1
 
Here are some examples of the API in action. A POST request is used for the query, in case the data we're sending is too large to fit in a GET request. Not it always uses POST (so not using the range of HTTP queries).

###Question/Answer dictionaries

I've organised the questions and answers to be in 4 value dictionaries, containing:

- dataset: lets the system know which class to instantiate etc, examples: postcode, census, movielens, ...etc
- dataitem: used by classes to know which aspect of the dataset. For example in the movielens dataset, one could be interested in whether the user's seen a film or what rating they've given the film.
- detail: often unused by the classes, could be, for example the id of the film we want to know about.
- answer: the user's answer.

###1. Get a suggestion for a question to answer *[action: question]*

This call returns a dictionary of {dataset,dataitem,detail,answer}.

'data' contains a list of previous asked (and answered) questions, to allow an optimum question to be asked.

Potentially will add 'features' as a key in the payload so you can choose what you want the question to maximise information about.

In [1]:
import requests
#Previous questions and answers are in 'data' as an array of dictionaries
data = [{'dataset':'postcode','dataitem':'postcode','detail':''}]
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'question'}
r = requests.post('http://scikic.org/api/api.cgi',json=payload)
r.content

'{"detail": "", "dataitem": "favourite_artist", "dataset": "music"}\n'

###2. Get a text string of the question *[action: questionstring]*

This API will change, as it will soon also return what buttons/etc will be required, to allow non-textual responses.

In [2]:
import requests

data = {'dataset':'postcode','dataitem':'postcode','detail':''}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'questionstring'}
r = requests.post('http://scikic.org/api/api.cgi',json=payload)
r.content

'"What\'s your postcode?"\n'

###3. Process Answer *[action: processanswer]*

Sometimes we need to improve or process an answer. For example if someone uses the 'where' class to give the city they are in, we can call 'processanswer' to get more details. To use, pass the question-answer dictionary as 'data' and use action 'processanswer'. The result is a new dictionary: It may be the same as the one you entered, or the class may change its contents. In this example it has looked up the latitude and longitude of sheffield.

In [3]:
import requests
data = {'dataset':'where','dataitem':'city','detail':'','answer':'sheffield'}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'processanswer'}
r = requests.post('http://scikic.org/api/api.cgi',json=payload)
r.content

'{"answer": "sheffield", "dataset": "where", "detail": "", "dataitem": "city"}\n'

###4. Inference *[action: inference]*

Put the questions and answers in 'data', as a list of dictionaries of {dataset,dataitem,detail,answer}.
Put a list of features that you want inference about in 'features' (for example ['factor_age','religion']).

Returns a dictionary of features, each one simply an array of the probabilities associated with each value. The API needs to provide a little more info here (TODO). If some probabilities are zero towards the end of a list then the list will be truncated. For example if inference is certain the user is a male, then the output list will be {"factor_gender":[1.0]}. If they are definitely female it will be {"factor_gender":[0.0, 1.0]}

In [4]:
import requests
data = [{'dataset':'postcode','dataitem':'postcode','detail':''}]
features = ['factor_age']
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'inference', 'features': features}
r = requests.post('http://scikic.org/api/api.cgi',json=payload)
r.content

'{"facts": {}, "features": {"religion": {"distribution": [0.7071111111111111, 0.0035555555555555557, 0.015555555555555555, 0.004, 0.04488888888888889, 0.005333333333333333, 0.0044444444444444444, 0.21511111111111111], "quartiles": {"upper": 4, "lower": 0, "mean": 1.7853333333333334}}, "factor_gender": {"distribution": [0.33155555555555555, 0.6684444444444444], "quartiles": {"upper": 1, "lower": 0, "mean": 0.6684444444444444}}, "oa": {"distribution": [1.0], "quartiles": {"upper": 0, "lower": 0, "mean": 0.0}}, "factor_age": {"distribution": [0.004888888888888889, 0.012888888888888889, 0.006222222222222222, 0.012444444444444444, 0.0071111111111111115, 0.010222222222222223, 0.006666666666666667, 0.010666666666666666, 0.009777777777777778, 0.010666666666666666, 0.007555555555555556, 0.010666666666666666, 0.012, 0.0044444444444444444, 0.008888888888888889, 0.008888888888888889, 0.008888888888888889, 0.011111111111111112, 0.010222222222222223, 0.013777777777777778, 0.008444444444444444, 0.009

###5. Metadata *[action: metadata]*

Some (ok, only one at the moment) of the classes provide metadata about the results. Use the 'metadata' action to retrieve these. Pass a dictionary in 'data' with the name of the dataset.

In [8]:
import requests
data = {'dataset':'census'}
payload = {"version":1, 'data': data, 'apikey': 'YOUR_API_KEY_HERE', 'action':'metadata'}
r = requests.post('http://scikic.org/api/api.cgi',json=payload)
r.content

'{"religions": ["Christian", "Buddhist", "Hindu", "Jewish", "Muslim", "Sikh", "Other religion", "No religion"], "religion_text": ["Christian", "Buddhist", "Hindu", "Jewish", "Muslim", "Sikh", "religious (but I do\'t know which)", "of no religion"]}\n'

###Insights

Some of the classes can provide insights based on the results of the inference (and some don't need the inference to be carried out and provide insights directly from the answers given).

Originally planned to have it as a seperate API query, but now returned with the inference (above).


###Future additions

- Get list of features available
- For inference, add 'features' as a key in the payload so you can choose what you want the question to maximise information about.
- Incorporate data: Need to be able to add new data for training.